In [3]:
# -*- coding: utf-8 -*-

import os
import pandas as pd
import numpy as np
from collections import Counter
import nltk
from nltk import sent_tokenize, word_tokenize
from string import punctuation
import string
from nltk.corpus import stopwords
stop = set(stopwords.words('russian'))
from nltk.stem import SnowballStemmer
from collections import Counter
stemmer = SnowballStemmer('russian')
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans


from random import shuffle

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/ahmad/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
variances = dict()

In [105]:
for item in med_dic['name']:
    df_temp = df[df['name_del_text'].str.contains(" " + item + " ")]
#     weights = df_temp['weight']
    prices = df_temp['price']
#     divided = prices.div(weights)
    try:
        variances[item] = np.nanmax(prices[prices!=np.inf])/np.nanmin(prices[prices!=-np.inf])
    except ValueError:
        pass

C:\Users\777\AppData\Local\conda\conda\envs\env_full\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: All-NaN axis encountered
  import sys


In [119]:
df[df['name_del_text'].str.contains(" тореал ")]

,_id,cont_num,sign_date,custom_regnum,product_name,okdp_code,okei_code,quantity,price,sum,name_norm,number,name_word,type,name_del,len,name_del_text,weight
272,13265695,0150200001612000043,2012-01-16,01502000016,Тореал таблетки покрытые оболочкой 100мг №28,2423219,778,50.0,543.74,27187.00,тореал таблетки покрытые оболочкой мг №,№28,"[тореал, таблетки, покрытые, оболочкой, мг, №]",tablet,[тореал],1,тореал,100.0
356,13426510,0180200002712000012,2012-01-19,01802000027,Тореал таблетки покрытые оболочкой 100 мг №28,2423210,778,132.0,565.95,74705.40,тореал таблетки покрытые оболочкой мг №,№28,"[тореал, таблетки, покрытые, оболочкой, мг, №]",tablet,[тореал],1,тореал,0.0
2784,18535910,0318200070712000039,2012-07-07,03182000707,Тореал\r\n,2423210,796,5.0,325.16,1625.80,тореал\r\n,,[тореал],unknown,[тореал],1,тореал,0.0
6518,22732651,0169200001012000803,2012-11-20,01692000010,Тореал\r\n,2423216,778,275.0,832.32,228888.00,тореал\r\n,,[тореал],unknown,[тореал],1,тореал,0.0
6519,22732652,0169200001012000803,2012-11-20,01692000010,Тореал\r\n,2423216,778,1.0,830.52,830.52,тореал\r\n,,[тореал],unknown,[тореал],1,тореал,0.0
6520,22732653,0169200001012000803,2012-11-20,01692000010,Тореал\r\n,2423216,778,40.0,150.30,6012.00,тореал\r\n,,[тореал],unknown,[тореал],1,тореал,0.0
9123,23334251,0160200005712000760,2012-12-03,01602000057,Топирамат\r\n\r\n \r\n\r\nТореал\r\n\r\nтаблет...,2423210,778,68.0,751.70,51115.60,топирамат\r\n\r\n \r\n\r\nтореал\r\n\r\nтаблет...,№28,"[топирамат, тореал, таблетки, покрытые, оболоч...",tablet,"[топирамат, тореал, лс, от]",4,топирамат тореал лс от,0.0
10576,23501054,0318300321412000136,2012-12-10,03183003214,Тореал таб. 25мг №28,2423219,778,2.0,252.62,505.24,тореал таб мг №,№28,"[тореал, таб, мг, №]",tablet,[тореал],1,тореал,25.0
11043,23624136,0188200000912000117,2012-12-10,01882000009,"Тореал, таблетки п/об 100 мг № 28\r\n",2423210,778,18.0,645.70,11622.60,тореал таблетки поб мг №\r\n,№28,"[тореал, таблетки, поб, мг, №]",tablet,[тореал],1,тореал,0.0
11262,23649862,0152200001512000346,2012-12-07,01522000015,Тореал\r\n\r\nтаблетки покрытые оболочкой 25 м...,2423210,778,300.0,138.27,41481.00,тореал\r\n\r\nтаблетки покрытые оболочкой мг ...,,"[тореал, таблетки, покрытые, оболочкой, мг, шт...",tablet,"[тореал, шт, банки, полимерные, пачки, картонные]",6,тореал шт банки полимерные пачки картонные,0.0


In [108]:
variances

{'плавикс': 1.1520224752742043,
 'амоксициллин': 5.153814089203835,
 'азитромицин': 3.613220998055735,
 'нитроксолин': 1.0,
 'офлоксацин': 1.2867372353673725,
 'флуконазол': 1.1411764705882352,
 'ципрофлоксацин': 82.73457943925234,
 'ремедиа': 1.2114747329303024,
 'ломацин': 1.0,
 'авелокс': 488.28965123370153,
 'норбактин': 1.0,
 'абактал': 4.83837037037037,
 'сульфасалазин': 1.3535852892256517,
 'сульпирид': 125803.085,
 'кардикет': 4.850953527124607,
 'пектрол': 1.0137083222782348,
 'актовегин': 111601.19247787612,
 'корвалол': 60438.496503496506,
 'семакс': 816319.51,
 'винпоцетин': 86034.3661971831,
 'аминазин': 1448788.8,
 'амитриптилин': 1991023.1951219514,
 'мексидол': 33230.39459815547,
 'церетон': 25898.326180257514,
 'дроперидол': 504437.39,
 'кавинтон': 14012.291794158553,
 'прозерин': 97997.49011857709,
 'фенорелаксан': 100.84033613445378,
 'фезипам': 30707.492795389047,
 'азилект': 1.948045856239563,
 'вимпат': 19.584055459272097,
 'галоперидол': -837.3280713091465,
 'гли

In [117]:
dict((k, v) for k, v in variances.items() if ((v < 10) and (v >= 5)))

{'амоксициллин': 5.153814089203835,
 'имован': 6.858,
 'пирлиндол': 6.520833333333333,
 'трилептал': 7.711498547114985,
 'тореал': 6.719844986274827,
 'бромокриптин': 5.632040050062578,
 'инстенон': 6.362324544142726,
 'эсмерон': 6.480245170971147,
 'пропофола': 9.2503259452412,
 'наропин': 8.044864864864865,
 'лоперамид': 7.373134328358209,
 'пантокальцин': 5.134153922665671,
 'спазган': 7.968,
 'тремонорм': 5.074648819592557,
 'бетамакс': 5.28,
 'баклофен': 6.4812353401094605,
 'бензобарбитал': 7.53,
 'пирацетамциннаризин': 6.288200000000001,
 'циталопрам': 9.011363636363637,
 'мексикор': 8.064516129032258,
 'транквезипам': 7.353170731707317,
 'катэна': 7.227818181818181,
 'паксил': 6.166592709984152,
 'тиодазин': 7.276041666666666,
 'топалепсин': 5.624433962264152,
 'велаксин': 5.705,
 'просульпин': 5.009174311926605,
 'санвал': 8.030150753768845,
 'золпидем': 7.945593220338982,
 'прокаин': 9.794117647058824,
 'промазин': 6.679538461538462,
 'зелдокс': 9.642410378178027,
 'галоперид

In [104]:
df[df['name_del_text'].str.contains(" тиоридазин ")]

,_id,cont_num,sign_date,custom_regnum,product_name,okdp_code,okei_code,quantity,price,sum,name_norm,number,name_word,type,name_del,len,name_del_text,weight
176,13193146,0107200000112000022,2012-01-10,01072000001,Тиоридазин,2423210,778,90.0,251.70,22653.00,тиоридазин,,[тиоридазин],unknown,[тиоридазин],1,тиоридазин,0.0
177,13193147,0107200000112000022,2012-01-10,01072000001,Тиоридазин,2423210,778,80.0,95.10,7608.00,тиоридазин,,[тиоридазин],unknown,[тиоридазин],1,тиоридазин,0.0
178,13193148,0107200000112000022,2012-01-10,01072000001,Тиоридазин,2423210,778,100.0,136.70,13670.00,тиоридазин,,[тиоридазин],unknown,[тиоридазин],1,тиоридазин,0.0
1046,14900853,0355300057212000027,2012-03-12,03553000572,Тиоридазин\r\n,2423210,778,30.0,262.68,7880.40,тиоридазин\r\n,,[тиоридазин],unknown,[тиоридазин],1,тиоридазин,0.0
1456,15441399,0387200001812000013,2012-03-26,03872000018,Тиоридазин,2423213,778,100.0,257.51,25751.00,тиоридазин,,[тиоридазин],unknown,[тиоридазин],1,тиоридазин,0.0
1457,15441400,0387200001812000013,2012-03-26,03872000018,Тиоридазин,2423213,778,200.0,413.49,82698.00,тиоридазин,,[тиоридазин],unknown,[тиоридазин],1,тиоридазин,0.0
1976,17054802,0111200002412000083,2012-05-17,01112000024,"Тиоридазин, таблетки покрытые пленочной оболоч...",2423213,778,1356.0,111.00,150516.00,тиоридазин таблетки покрытые пленочной оболочк...,№100,"[тиоридазин, таблетки, покрытые, пленочной, об...",tablet,"[тиоридазин, тиорил, пплен, об]",4,тиоридазин тиорил пплен об,25.0
2219,17338888,0334200003112000011,2012-05-28,03342000031,Тиоридазин драже 10 мг №60\r\n,2423213,778,50.0,246.73,12336.50,тиоридазин драже мг №\r\n,№60,"[тиоридазин, драже, мг, №]",tablet,[тиоридазин],1,тиоридазин,0.0
2220,17338889,0334200003112000011,2012-05-28,03342000031,Тиоридазин драже 25 мг №60\r\n,2423213,778,720.0,392.26,282427.20,тиоридазин драже мг №\r\n,№60,"[тиоридазин, драже, мг, №]",tablet,[тиоридазин],1,тиоридазин,0.0
2629,18148570,0366200011412000029,2012-06-26,03662000114,Тиоридазин\r\n,2423213,778,6.0,267.74,1606.44,тиоридазин\r\n,,[тиоридазин],unknown,[тиоридазин],1,тиоридазин,0.0


In [13]:
type(med_dic)

pandas.core.frame.DataFrame

In [14]:
med_dic.head(30)

,name,class
index,,
0,плавикс,0
1,амоксициллин,1
2,азитромицин,2
3,метронидазол,3
4,нитроксолин,4
5,офлоксацин,5
6,флуконазол,6
7,ципрофлоксацин,7
8,ремедиа,8


In [16]:


col_type={'_id': str,
           'cont_num':str,
           'sign_date':str,
           'custom_regnum':str,
           'product_name':str,
           'okdp_code':str,
           'okei_code':str,
           'quantity':float,
           'price':float,
           'sum':float
           }

df = pd.read_csv('medical_2012.csv', sep=';',dtype=col_type)

df['name_norm']=df['product_name'].map(lambda x: re.sub(r'(\D)\.', r'\1. ',x))  # delete symbol point everywhere except for in numbers
df['name_norm']=df['name_norm'].map(lambda x: x.lower())
df['name_norm']=df['name_norm'].map(lambda x: x.replace('№ ','№')) 
df['name_norm']=df['name_norm'].map(lambda x: re.sub(r' n(\d+)| n (\d+)| no(\d+)| no (\d+)', r' №\1',x)) 

mystoplist=[' для ',' д/', ' и ', ' в ',' с ',' у ', ' на ', ' а ',' не ', ' о '] # need also exclude \\dlya
my_regex = re.compile('|'.join(mystoplist))
df['name_norm']=df['name_norm'].apply(lambda x: my_regex.sub(' ',x)) 
df['name_norm']=df['name_norm'].apply(lambda x: x.replace(':',' ')) 
#punctuation=''.join([sim for sim in list(punctuation) if sim not in ['%',',']]) # list of simbols not to delete

translator = str.maketrans('','',punctuation) 
df['name_norm']=df['name_norm'].map(lambda x: x.translate(translator))

def prod_number(doc):
    substr=re.search('№\d+',doc)
    if substr: return str(substr[0])
    return ''

df['number']=df['name_norm'].apply(prod_number)

# comment it later. it deletes all numbers
df['name_norm']=df['name_norm'].map(lambda x: re.sub('\d+', '',x))  

df['name_word']=df['name_norm'].map(lambda x : word_tokenize(x))

type_dict={'tablet':['таблетки', 'таблетка', 'таб', 'таблет', 'тб', 'тбт', 'табл', 'драже', 'др', 'драж', 'капс', 'капсула', 'капсулы'],
           'cream':['мазь', 'мз', 'мзь', 'эмульс', 'эмульсия'],
           'powder':['пор', 'порошок'],
           'liquid':['рр','рас','раст', 'раств', 'раствор', 'концентрат', 'конц', 'сусп', 'сироп', 'сир'],
           'capsule':['амп', 'ампулы', 'ампула'],
           'spray': ['спрей', 'спр', 'аэр', 'аэрозоль', 'аэроз'],
           'dropley':['капли', 'каплях', 'капл', 'кап']           
           }
# import pycountry # use it later
countr_list=['австрия','франция','польша','россия','украина', 'латвия', 'мальта', 'хорватия'] # use it later

def prod_type(doc):
    for key, value in type_dict.items():    
        if len([w for w in doc if w in value])>0:
            return key
    return 'unknown'            

df['type']=df['name_word'].apply(prod_type)

freqs = Counter(df['type'])
freqs.most_common()


def del_list(doc,dic):
    text=doc
    for key, value in dic.items():    
        text=[w for w in text if w not in value]
    return text

df['name_del']=df['name_word'].apply(del_list, dic=type_dict)

okdp_list=[str(okdp) for okdp in range(2423210,2423220)] # modify for more products

del_dict={'okdp':okdp_list,
          'word1':['средства', 'противосудорожные','действующие', 'центральную', 'нервную', 'снотворные', 'наркоза', 'систему', 'прочие', 'препараты', 'влияющие', ','],
          'word2':['покрытые','внутримышечного','возбуждающие','влияющих','нейроплегические','лекарственный','средствадействующие','лекарственные','лекарственных','н','пролонгированного','наркотические','седативные','','средств','введения','внутривенного','инъекций','оболочкой','по','поб','пленочной','действия','внутрь', 'приема', 'введ', 'поставка'],
          'help':['вв','вм','мгмл','мг','мл','г', '№','мс'], # delete this category later
          'counry': countr_list # delete this category later
          }

df['name_del']=df['name_del'].apply(del_list, dic=del_dict)

df['len']=df['name_del'].apply(lambda x: len(x))
df=df[df['len']>0]

# classification of one-word prodcuts
df_len1=df[df['len']==1]
df_len1['name']=df_len1['name_del'].apply(lambda x: x[0])

med_dic=df_len1.loc[:,'name']
med_dic=pd.DataFrame(med_dic.drop_duplicates())
med_dic['class']=range(0,len(med_dic['name']))
med_dic['index']=med_dic['class']
med_dic=med_dic.set_index('index')

df_len1=pd.merge(df_len1, med_dic, on='name', how='left')

# classification of more-than-one-word prodcuts w.r.t. one-word-products
df_len2=df[df['len']>1]

def search_list(doc):
    for w in doc:
        ind=[i for i, j in enumerate(med_dic['name']) if j == w]    
        if len(ind)>0:
            return med_dic.loc[ind[0],'class']
    return 

df_len2['class']=df_len2['name_del'].apply(search_list)
df_len2=pd.merge(df_len2, med_dic, on='class', how='left')
# Check classes if needed
#df_len2['class'].isna().sum()
#freqs = Counter(df_len2['class'])
#pop_class=[cl[0] for cl in freqs.most_common()[0:50]]

df_final=pd.concat([df_len1,df_len2])
df_final=df_final[df_final['class'].isna()==0]
df_final['number']=df_final['number'].apply(lambda x: x.replace('№',''))
df_final=df_final.drop(['name','name_del','name_word','product_name'], axis=1)
df_final['year']=df_final['sign_date'].apply(lambda x: x[:4])
df_final['month']=df_final['sign_date'].apply(lambda x: x[5:7])


import csv
df_final.to_csv('medical_2012_class.csv', sep=';',quoting=csv.QUOTE_NONNUMERIC)





# classify all who is not-classified through k-mean
# this is old version. I did not execute this part of code in the final version
# corp=[]
# for w in df_len2.loc[df_len2['class'].isna(),'name_del']:
#     corp+=w
# freqs = Counter(corp)
# freqs.most_common()[:100]

# df['name_del_text']=df['name_del'].apply(lambda x: ' ' + ' '.join(x) + ' ')

# df1=df[df['type']=='tablet']
# vec = CountVectorizer(max_features=600) 
# X = vec.fit_transform(df1['name_del_text'])
# vocabdict = vec.vocabulary_ # it has type vocabulary

# km = KMeans(n_clusters=500,n_jobs=-1)
# km.fit(X)
# doc_clusters = km.labels_.tolist()
# df1['cluster'] = doc_clusters

# freqs = Counter(df1['cluster'])
# freqs.most_common()[:20]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:117: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept 

In [17]:
df_final.head(30)

,_id,class,cont_num,custom_regnum,len,name_norm,number,okdp_code,okei_code,price,quantity,sign_date,sum,type,year,month
0,12798708,0.0,0318300093212000003,03183000932,1,плавикс таб по мг №,10,2423219,778,2308.63,6.0,2012-01-03,13851.78,tablet,2012,01
1,12799474,1.0,0318300093212000005,03183000932,1,амоксициллин таб мг №,20,2423219,778,23.99,150.0,2012-01-03,3598.50,tablet,2012,01
2,12799359,2.0,0318300093212000005,03183000932,1,азитромицин капс мг №,6,2423219,778,102.64,480.0,2012-01-03,49267.20,tablet,2012,01
3,12799475,1.0,0318300093212000005,03183000932,1,амоксициллин таб мг №,20,2423210,778,44.29,150.0,2012-01-03,6643.50,tablet,2012,01
4,12799555,3.0,0318300093212000005,03183000932,1,метронидазол таб №,20,2423219,778,6.83,300.0,2012-01-03,2049.00,tablet,2012,01
5,12799556,4.0,0318300093212000005,03183000932,1,нитроксолин таб по г №,50,2423219,778,34.35,30.0,2012-01-03,1030.50,tablet,2012,01
6,12799557,5.0,0318300093212000005,03183000932,1,офлоксацин таб по мг №,10,2423219,778,41.33,35.0,2012-01-03,1446.55,tablet,2012,01
7,12799559,6.0,0318300093212000005,03183000932,1,флуконазол капс мг №,1,2423219,778,19.40,60.0,2012-01-03,1164.00,tablet,2012,01
8,12799590,7.0,0318300093212000005,03183000932,1,ципрофлоксацин таб по мг №,10,2423219,778,19.66,260.0,2012-01-03,5111.60,tablet,2012,01
9,12799591,7.0,0318300093212000005,03183000932,1,ципрофлоксацин таб по мг №,10,2423219,778,10.70,50.0,2012-01-03,535.00,tablet,2012,01


In [7]:
mystr = '0,05г'
substr=re.search('\d+,\d+г',mystr)
substr[0]

'0,05г'

In [8]:
def convert_float(num):
    return float(num.replace(',', '.'))

In [9]:
def weight_mg(doc):
    substr=re.search('\d+,\d+мг',doc)
    if substr: return convert_float(str(substr[0])[:-2])
    substr=re.search('\d+\.\d+мг',doc)
    if substr: return convert_float(str(substr[0])[:-2])
    substr=re.search('\d+мг',doc)
    if substr: return convert_float(str(substr[0])[:-2])
    return 0

In [10]:
def weight_g(doc):
    substr=re.search('\d+,\d+г',doc)
    if substr: return convert_float(str(substr[0])[:-1])
    substr=re.search('\d+\.\d+г',doc)
    if substr: return convert_float(str(substr[0])[:-1])
    substr=re.search('\d+г',doc)
    if substr: return convert_float(str(substr[0])[:-1])
    return 0

In [11]:
df['weight']=df['product_name'].apply(weight_mg) + 1000 * df['product_name'].apply(weight_g)

In [12]:
df.head(30)

,_id,cont_num,sign_date,custom_regnum,product_name,okdp_code,okei_code,quantity,price,sum,name_norm,number,name_word,type,name_del,len,name_del_text,weight
0,12798676,0318300093212000003,2012-01-03,03183000932,Гидрокортизон-Рихтер сусп для в/м и внутрисуст...,2423219,778,40.0,84.53,3381.20,гидрокортизонрихтер сусп вм внутрисуст вв мгмл...,№1,"[гидрокортизонрихтер, сусп, вм, внутрисуст, вв...",liquid,"[гидрокортизонрихтер, внутрисуст]",2,гидрокортизонрихтер внутрисуст,25.0
1,12798677,0318300093212000003,2012-01-03,03183000932,Лопирел таб. п/об пленочн. 75мг №28,2423219,778,90.0,689.57,62061.30,лопирел таб поб пленочн мг №,№28,"[лопирел, таб, поб, пленочн, мг, №]",tablet,"[лопирел, пленочн]",2,лопирел пленочн,75.0
2,12798708,0318300093212000003,2012-01-03,03183000932,Плавикс таб. п/о 300мг №10,2423219,778,6.0,2308.63,13851.78,плавикс таб по мг №,№10,"[плавикс, таб, по, мг, №]",tablet,[плавикс],1,плавикс,300.0
3,12798709,0318300093212000003,2012-01-03,03183000932,"Фраксипарин р-р для п/к введ. 9,5тыс. анти-Ха ...",2423219,778,10.0,1484.56,14845.60,фраксипарин рр пк введ тыс антиха мемл шприц...,№10,"[фраксипарин, рр, пк, введ, тыс, антиха, мемл,...",liquid,"[фраксипарин, пк, тыс, антиха, мемл, шприц, од...",9,фраксипарин пк тыс антиха мемл шприц однодозо...,0.0
4,12798710,0318300093212000003,2012-01-03,03183000932,Тромбофлюкс лиоф. пригот. р-ра в/в и в/а введ....,2423219,778,20.0,8472.64,169452.80,тромбофлюкс лиоф пригот рра вв ва введ млн ...,№1,"[тромбофлюкс, лиоф, пригот, рра, вв, ва, введ,...",unknown,"[тромбофлюкс, лиоф, пригот, рра, ва, млн, ме, фл]",8,тромбофлюкс лиоф пригот рра ва млн ме фл,0.0
5,12799474,0318300093212000005,2012-01-03,03183000932,Амоксициллин таб. 250мг №20,2423219,778,150.0,23.99,3598.50,амоксициллин таб мг №,№20,"[амоксициллин, таб, мг, №]",tablet,[амоксициллин],1,амоксициллин,250.0
6,12799359,0318300093212000005,2012-01-03,03183000932,Азитромицин капс. 250мг №6,2423219,778,480.0,102.64,49267.20,азитромицин капс мг №,№6,"[азитромицин, капс, мг, №]",tablet,[азитромицин],1,азитромицин,250.0
7,12799473,0318300093212000005,2012-01-03,03183000932,Амикацин-Виал пор. пригот. р-ра в/в и в/м введ...,2423219,778,2000.0,32.28,64560.00,амикацинвиал пор пригот рра вв вм введ фл ...,№1,"[амикацинвиал, пор, пригот, рра, вв, вм, введ,...",powder,"[амикацинвиал, пригот, рра, фл]",4,амикацинвиал пригот рра фл,500.0
8,12799475,0318300093212000005,2012-01-03,03183000932,Амоксициллин таб. 500мг №20,2423210,778,150.0,44.29,6643.50,амоксициллин таб мг №,№20,"[амоксициллин, таб, мг, №]",tablet,[амоксициллин],1,амоксициллин,500.0
9,12799476,0318300093212000005,2012-01-03,03183000932,Ампициллин пор. пригот. р-ра в/в и в/м введ. 1...,2423219,778,700.0,9.11,6377.00,ампициллин пор пригот рра вв вм введ мг фл №,№1,"[ампициллин, пор, пригот, рра, вв, вм, введ, м...",powder,"[ампициллин, пригот, рра, фл]",4,ампициллин пригот рра фл,1000.0


In [99]:
df[df['name_del_text'].str.contains(" авелокс ")]

,_id,cont_num,sign_date,custom_regnum,product_name,okdp_code,okei_code,quantity,price,sum,name_norm,number,name_word,type,name_del,len,name_del_text,weight
27,12800173,0318300093212000006,2012-01-03,03183000932,"Авелокс р-р д/инф. 1,6мг/мл фл. 250мл",2423219,778,10.0,2505.93,25059.30,авелокс рр инф мгмл фл мл,,"[авелокс, рр, инф, мгмл, фл, мл]",liquid,"[авелокс, инф, фл]",3,авелокс инф фл,1.6
28,12800174,0318300093212000006,2012-01-03,03183000932,Авелокс таб. п/о 400мг №5,2423219,778,40.0,1033.07,41322.80,авелокс таб по мг №,№5,"[авелокс, таб, по, мг, №]",tablet,[авелокс],1,авелокс,400.0
22009,3763387_7,0318300093212000006,2012-01-03,03183000932,"Авелокс р-р д/инф. 1,6мг/мл фл. 250мл",2423219,778,10.0,2507.77,25077.70,авелокс рр инф мгмл фл мл,,"[авелокс, рр, инф, мгмл, фл, мл]",liquid,"[авелокс, инф, фл]",3,авелокс инф фл,1.6
22010,3763387_8,0318300093212000006,2012-01-03,03183000932,Авелокс таб. п/о 400мг №5,2423219,778,40.0,1033.07,41322.80,авелокс таб по мг №,№5,"[авелокс, таб, по, мг, №]",tablet,[авелокс],1,авелокс,400.0
29312,4128898_25,0373100047412000064,2012-02-01,03731000474,Авелокс,2423219,778,5.0,1040.05,5200.25,авелокс,,[авелокс],unknown,[авелокс],1,авелокс,0.0
29313,4128898_26,0373100047412000064,2012-02-01,03731000474,Авелокс,2423219,778,50.0,2398.55,119927.50,авелокс,,[авелокс],unknown,[авелокс],1,авелокс,0.0
47944,4777756_7,0318300093212000006,2012-01-03,03183000932,"Авелокс р-р д/инф. 1,6мг/мл фл. 250мл",2423219,778,10.0,2505.93,25059.30,авелокс рр инф мгмл фл мл,,"[авелокс, рр, инф, мгмл, фл, мл]",liquid,"[авелокс, инф, фл]",3,авелокс инф фл,1.6
47945,4777756_8,0318300093212000006,2012-01-03,03183000932,Авелокс таб. п/о 400мг №5,2423219,778,40.0,1033.07,41322.80,авелокс таб по мг №,№5,"[авелокс, таб, по, мг, №]",tablet,[авелокс],1,авелокс,400.0
81407,5871785_2,0348300118212000117,2012-07-11,03483001182,"АРО 2423219,2423475,2423710,2423245,2423360,24...",2423219,383,1.0,504437.39,504437.39,аро ношбра авелокс актовегин актрапид сульфа...,,"[аро, ношбра, авелокс, актовегин, актрапид, су...",unknown,"[аро, ношбра, авелокс, актовегин, актрапид, су...",57,аро ношбра авелокс актовегин актрапид сульфац...,0.0
101569,6471235_25,0373100047412000286,2012-09-10,03731000474,Авелокс,2423219,778,5.0,1039.94,5199.70,авелокс,,[авелокс],unknown,[авелокс],1,авелокс,0.0


In [3]:
df_final.head(10)

,_id,class,cont_num,custom_regnum,len,number,okdp_code,okei_code,price,quantity,sign_date,sum,type,year,month
0,12798708,0.0,0318300093212000003,03183000932,1,10,2423219,778,2308.63,6.0,2012-01-03,13851.78,tablet,2012,01
1,12799474,1.0,0318300093212000005,03183000932,1,20,2423219,778,23.99,150.0,2012-01-03,3598.50,tablet,2012,01
2,12799359,2.0,0318300093212000005,03183000932,1,6,2423219,778,102.64,480.0,2012-01-03,49267.20,tablet,2012,01
3,12799475,1.0,0318300093212000005,03183000932,1,20,2423210,778,44.29,150.0,2012-01-03,6643.50,tablet,2012,01
4,12799555,3.0,0318300093212000005,03183000932,1,20,2423219,778,6.83,300.0,2012-01-03,2049.00,tablet,2012,01
5,12799556,4.0,0318300093212000005,03183000932,1,50,2423219,778,34.35,30.0,2012-01-03,1030.50,tablet,2012,01
6,12799557,5.0,0318300093212000005,03183000932,1,10,2423219,778,41.33,35.0,2012-01-03,1446.55,tablet,2012,01
7,12799559,6.0,0318300093212000005,03183000932,1,1,2423219,778,19.40,60.0,2012-01-03,1164.00,tablet,2012,01
8,12799590,7.0,0318300093212000005,03183000932,1,10,2423219,778,19.66,260.0,2012-01-03,5111.60,tablet,2012,01
9,12799591,7.0,0318300093212000005,03183000932,1,10,2423219,778,10.70,50.0,2012-01-03,535.00,tablet,2012,01


In [2]:
!pip install nltk

  Running setup.py bdist_wheel for nltk: started
  Running setup.py bdist_wheel for nltk: finished with status 'done'
  Stored in directory: C:\Users\777\AppData\Local\pip\Cache\wheels\4b\c8\24\b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built nltk


You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\777\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
!pip install utils

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\777\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True